# Introducción a Kivy

Kivy es un framework multiplataforma de código abierto para la creación de interfaces gráficas de usuario. Está especialmente dirigido a plataformas móviles (Android e iOS), pero también puede ejecutarse en Windows. Linux y macOS.

Una aplicación creada con el framework kivy puede empaquetarse con pyinstaller para ejecutarse en equipos de escritorio, pero además el proyecto kivy incluye subproyectos que permiten distribuir las aplicaciones creadas con el framework kivy en Android e iOS.

## INSTALACIÓN EN LINUX


+ **1 - Instalar kivy**

+ Instalar virtualenv en python3

+ Crear un entorno virtual llamado kivyenv

+ Activar el entorno kivyenv (cambiar a ruta del entorno)

+ Instalar kivy en nuestro kivyenv

+ **2 - Instalar buildozer**

+ **Instalar buildozer en nuestro kivyenv**

buildozer se encargará de crear nuestros apks instalables en dispositivos android. Para funcionar correctamente es necesario instalar, además, cython y virtualenv en nuestro kivyenv. Por otra parte, tenemos que instalar el jdk8 para que buildozer pueda compilar el archivo apk. Los comandos para realizar la instalación completa son los siguientes:

+ **Configurar JAVA_HOME y el PATH del sistema:**

Abre el archivo ~/.bashrc con cualquier editor de texto y al final del archivo añade lo siguiente:

Si no añades al archivo .bashrc la variable de entorno JAVA_HOME puede que buildozer funcione correctamente, pero si en tu equipo tienes instaladas otras versiones del java development kit (jdk) y/o android studio puede que cuando buildozer ejecute javac esté utilizando una versión diferente e incompatible del jdk y la compilación falle (esto me dió muchos quebraderos de cabeza).
En cualquier caso, siempre debes añadir export PATH=$PATH:~/.local/bin.

Finalmente ejecuta source ~/.bashrc para aplicar los cambios sin reiniciar el equipo.

**NOTAS**

Hemos instalado kivy y buildozer en un virtualenv llamado kivyenv. Hay varios motivos por los que hemos realizado la instalación en un entorno virtual y no directamente en la instalación de python3 de nuestro equipo:

+ kivy tiene muchas dependencias que pueden corromperse al instalar otros módulos y paquetes en nuestra instalación de python3.
+ pyinstaller generalmente crea ejecutables de mayor tamaño cuando se trabaja fuera de un virtualenv, debido al mecanismo de búsqueda de dependencias entre módulos y paquetes.
+ Aunque la documentación oficial de kivy recomienda realizar la instalación en un virtualenv lo cierto es que he intentado instalarlo en la instalación de python3 del equipo y no he tenido éxito.

Si has prestado atención a los comandos ejecutados para instalar buildozer te habrás fijado en que, entre otras cosas, buildozer depende de Cython. Por otra parte, dentro de nuestro kivyenv hemos instalado virtualenv: esto aunque parezca "redundante" es necesario, ya que buildozer creará un entorno virtual para nuestra aplicación, que compilará para la plataforma móvil deseada (iOS o Android).

No es necesario instalar pyjnius ni plyer, ya que buildozer se encargará de instalarlos si son necesarios (vía pip) en el momento de crear la distribución para la plataforma seleccionada. Tampoco tenemos que preocuparnos de instalar python-for-android: buildozer se encarga de todo.




### Ejemplo: APK para Android (kivy + pyjnius)

Vamos a crear un proyecto llamado ejemplo_kivy:



En nuestro proyecto de ejemplo hemos creado dos archivos (main.py y second.py). En main.py crearemos una sencilla interfaz gráfica y en second.py definiremos las funciones manejadoras de eventos.

**main.py**

In [ ]:
from kivy.app import App
from kivy.uix.boxlayout import BoxLayout
from kivy.uix.button import Button


class btnToast(Button):
    text = 'Toast'
    def on_press(self):
        print('QUIERO MOSTRAR UN TOAST')

class btnShare(Button):
    text = 'Share'
    def on_press(self):
        print('QUIERO COMPARTIR UN TEXTO POR WHATSAPP')

class Box(BoxLayout):
    orientation = 'vertical'
    def __init__(self):
        super(Box, self).__init__()
        self.add_widget(btnToast())
        self.add_widget(btnShare())

class MainApp(App):
    def build(self):
        return Box()


if __name__ == "__main__":
    MainApp().run()

**second.py**

In [ ]:
from kivy.utils import platform

if platform == 'android':

    from jnius import autoclass, cast
    from android.runnable import run_on_ui_thread

    context = autoclass('org.renpy.android.PythonActivity').mActivity
    Toast = autoclass('android.widget.Toast')
    String = autoclass('java.lang.String')
    Intent = autoclass('android.content.Intent') 

    def charseq(text):
        return cast('java.lang.CharSequence', String(text))


    @run_on_ui_thread
    def toast(text):

        text = charseq(text)
        mi_toast = Toast.makeText(context, text, Toast.LENGTH_SHORT)
        mi_toast.show()


    @run_on_ui_thread
    def share(text):

        text, subject = charseq(text), charseq(text.replace(' ', '-'))

        intent = Intent()
        intent.setAction(Intent.ACTION_SEND)

        intent.putExtra(Intent.EXTRA_SUBJECT, subject)
        intent.putExtra(Intent.EXTRA_TEXT, text)

        intent.setType('text/plain')

        context.startActivity(intent)

else:
    toast = print
    share = print